# Data Setup

Before running this notebook, update the `config.py` file with your catalog and schema names.  
A dedicated compute cluster is required to run this notebook:  
- Use **Approach 1** with Classic Compute  
- Use **Approach 2** with Serverless


Run this cell to initialize resource names. You can edit resource names in the config file before running.

In [0]:
%run ./config

DataFrame[]

## Write data from CSV to schema to use with AI funcions

In [0]:
import pandas as pd
df = pd.read_csv('./data/synthetic_car_data.csv')
spark_df = spark.createDataFrame(df)
spark_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.synthetic_car_data")

## Write data from PNG to schema to use with AI funcions

In [0]:
import os

current_dir = os.path.dirname(dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get())
data_path = f"file:/Workspace{current_dir}/data"
display(data_path)
file_path = f"/Volumes/{catalog_name}/{schema_name}/dip/dip.png"

'file:/Workspace/Users/respinozahe@alumni.unav.es/Agent-Bricks-Workshop/data'

## Write data from ZIP file to use with Agent Bricks 

### Approach #1: Clone and Move Data from GitHub

You need a classic compute cluster (not serverless) to move files from the workspace to a UC volume.


In [0]:
display(f"/Workspace{current_dir}/data/tech_support.zip")

'/Workspace/Users/respinozahe@alumni.unav.es/Agent-Bricks-Workshop/data/tech_support.zip'

In [0]:
import zipfile
import os
import shutil

# Tus rutas actuales
current_dir = os.path.dirname(dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get())

# 1. Descomprimir en /Workspace
zip_path = f"/Workspace{current_dir}/data/tech_support.zip"
extract_path = f"/Workspace{current_dir}/data/tech_support/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
    print(f"Archivos descomprimidos en: {extract_path}")

# 2. Limpiar archivos .DS_Store
def delete_ds_store_files(root_dir):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename == '.DS_Store':
                file_path = os.path.join(dirpath, filename)
                try:
                    os.remove(file_path)
                except Exception as e:
                    print(f"Could not delete {file_path}: {e}")

delete_ds_store_files(extract_path)

# 3. Crear volumen
spark.sql(f"CREATE VOLUME IF NOT EXISTS {catalog_name}.{schema_name}.`tech_support`")

# 4. Copiar directamente de /Workspace a /Volumes usando Python
source_path = extract_path
target_path = f"/Volumes/{catalog_name}/{schema_name}/tech_support/"

# Crear directorio destino si no existe
os.makedirs(target_path, exist_ok=True)

# Copiar recursivamente
shutil.copytree(source_path, target_path, dirs_exist_ok=True)

print(f"Archivos copiados exitosamente a: {target_path}")

# 5. Verificar archivos copiados
files = dbutils.fs.ls(f"/Volumes/{catalog_name}/{schema_name}/tech_support/")
print(f"\nArchivos en el volumen ({len(files)} elementos):")
for f in files[:5]:  # Mostrar primeros 5
    print(f"  - {f.name}")

Archivos descomprimidos en: /Workspace/Users/respinozahe@alumni.unav.es/Agent-Bricks-Workshop/data/tech_support/
Archivos copiados exitosamente a: /Volumes/agent_bricks_demo/data_schema/tech_support/

Archivos en el volumen (4 elementos):
  - __MACOSX/
  - knowledge_base/
  - support_tickets/
  - tech_support/


### Approach #2: Uploading Data Directly to a Unity Catalog Volume

1. Download the Data </br>
Download the tech_support.zip archive from the provided GitHub [link](https://github.com/chen-data-ai/Agent-Bricks-Workshop/blob/main/data/tech_support.zip).

2. Upload the ZIP File </br>
Go to your Databricks workspace.
Navigate to your schema’s Unity Catalog volume using one of these methods:

    - Sidebar: Select Add data > Upload files to volume
    - Catalog Explorer: Click Add > Upload to volume
    - Notebook: Select File > Upload the downloaded tech_support.zip file and your catalog or schema information

3. Copy the File Path </br>
After uploading, locate the file path (e.g., /Volumes/catalog/schema/volume/tech_support.zip) — you’ll need this path for the next step.

4. Unzip the File in Databricks </br>
Use the following command in a notebook cell to unzip the file to your desired directory:

In [0]:
# %sh unzip '/Volumes/<ADD YOUR CATALOG>/<ADD YOUR SCHEMA>/tech_support/tech_support.zip' -d '/Volumes/<ADD YOUR CATALOG>/<ADD YOUR SCHEMA>/tech_support'